In [1]:
import spacy
from spacy import displacy

In [22]:
nlp = spacy.load('en_core_web_sm')

In [64]:
doc = nlp(u'The girl stood there, where she was, and tried to put in order the shabby hair, extremely angry with the fact that it saw drivers who were waiting for the move.')
displacy.render(doc, style='dep', jupyter=True)

In [58]:
x=doc[1]
x.

'VBP'